In [ ]:
import tensorflow as tf
import numpy as np
import math

%matplotlib inline
%load_ext autoreload
%autoreload 2

In [ ]:
from util import import_dataset

address = '../../data/'
file_names = {}
file_names['images'] = 'full_data.npy'
file_names['subs'] = 'full_subredditlabels'
file_names['dict'] = 'full_subredditIndex'
file_names['nsfw'] = 'full_nsfwlabels'
data, dictionary = import_dataset(address, file_names)

In [ ]:
print(data.X_train.shape)
print(data.X_val.shape)
print(data.X_test.shape)
print(data.y_train.shape)
print(data.y_val.shape)
print(data.y_test.shape)

### Balancing dataset

In [1]:
from util import sample_data

#dictionary = {j:i for i, j in dictionary.items()}
subreddits = ['gonewild', 'ladybonersgw', 'PrettyGirls', 'LadyBoners']
dictionary_2 = sample_data(subreddits, data, dictionary)

NameError: name 'data' is not defined

ERROR:tornado.application:Exception in callback functools.partial(<function wrap.<locals>.null_wrapper at 0x7ffb789342f0>)
Traceback (most recent call last):
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/tornado/ioloop.py", line 600, in _run_callback
    ret = callback()
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/tornado/stack_context.py", line 275, in null_wrapper
    return fn(*args, **kwargs)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/kernelbase.py", line 242, in enter_eventloop
    self.eventloop(self)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/eventloops.py", line 163, in loop_tk
    kernel.timer = Timer(doi)
  File "/home/cs231n/myVE35/lib/python3.5/site-packages/ipykernel/eventloops.py", line 151, in __init__
    self.app = Tk()
  File "/usr/lib/python3.5/tkinter/__init__.py", line 1871, in __init__
    self.tk = _tkinter.create(screenName, baseName, className, interactive, wantobjects, useTk, sync, use)
_tkint

In [ ]:
from googlenet import GoogleNetMulticlass
from config import ModelConfig, TrainConfig

tf.reset_default_graph()

model_config = ModelConfig(eval_batch_size=512, learning_rate=5e-4, output="multiclass")
train_config = TrainConfig(print_every=20, num_epochs=15)
model = GoogleNet(model_config)
sess = tf.Session()
model.train(data, sess, train_config)

In [ ]:
model.plot_loss_acc(data)